In [1]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
os.environ["GROQ_API_KEY"] = getpass.getpass()

 ········
 ········


In [21]:
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage,AIMessage
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [3]:
# Instantiating the Model
model = ChatGroq(model="llama3-8b-8192")

In [11]:
model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='I know this one! Your name is Bob!', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 40, 'total_tokens': 51, 'completion_time': 0.009166667, 'prompt_time': 0.006713017, 'queue_time': 0.006908543, 'total_time': 0.015879684}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-8a8c8021-5788-4d9d-aebd-a792572c4ba4-0', usage_metadata={'input_tokens': 40, 'output_tokens': 11, 'total_tokens': 51})

In [17]:
# Initializing a dictionary
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)
config = {"configurable": {"session_id": "abc2"}}

In [18]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Yuvraj")],
    config=config,
)

response.content

"Nice to meet you, Yuvraj! How's your day going so far?"

In [19]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Yuvraj!'

In [22]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [23]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm Yuvraj")]})

response.content

"Hi Yuvraj! Nice to meet you! How can I assist you today? Do you have a question, need help with something, or just want to chat? I'm all ears!"

In [24]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)
config = {"configurable": {"session_id": "abc4"}}

In [25]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Yuvraj")],
    config=config,
)

response.content

"Nice to meet you, Yuvraj! How can I assist you today? Do you have a question, need help with something, or just want to chat? I'm all ears!"